In [2]:
!pip install boto3

In [3]:
import boto3
s3 = boto3.resource('s3')
def s3_bucket_object_keys(bucket_name= 'iia-vault-telemetry-practice-unzipped'):
    bucket = s3.Bucket(bucket_name)
    key_list=[]
    for key in bucket.objects.all():
        key_list.append(key)
    return(key_list)

In [4]:
from io import BytesIO
def load_mat_file(s3_key, bucketname= 'iia-vault-telemetry-practice-unzipped'):
    s3 = boto3.resource('s3')
    obj = s3.Object(bucketname, s3_key)
    inMATFile = obj.get()['Body'].read()
    raw_mat = io.loadmat(BytesIO(inMATFile))
    return(raw_mat)

In [5]:
start = datetime.datetime.now()
key_list = s3_bucket_object_keys()
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:18.019696


In [1]:
from scipy import io
import numpy as np
import csv
import matplotlib as mp
import matplotlib.pyplot as plt
from numpy.lib import recfunctions
import datetime
import numpy.lib.recfunctions as recfn
import pandas as pd
import glob
import json

# # Open flight .mat file as a python dictionary
# def open_mat_file(file_name):
#     #start = datetime.datetime.now()
#     inMATFile = file_name
#     raw_data = io.loadmat(inMATFile)
#     #end = datetime.datetime.now()
#     #print("Time Taken to run the open_mat_file ",str(end-start))
#     return raw_data


# Extract the names of the features (i.e. the keys in the raw python dictionary)
# starting point: 189 features
# Delete 3 specific keys since they are metadata and not associated with the blackbox data
# intermediate total: 186
# Delete all features that have resolution < 1Hz since these are  static and non-relevant
# Final total: 163

def extract_features(raw_data):
    
    features = raw_data.keys()
    features_25 = []
    features_1 = []
    data_def=[]
    number_of_features =  len(features) 
    #print number_of_features
#     del features[137]
#     del features[160]
#     del features[181]
    for key in features:
        if key == '__header__' or key == '__version__' or key == '__globals__':
            continue
        data =  raw_data[key]['data'][0][0].T
        rate =  raw_data[key]['Rate'][0][0][0][0]
        try:
            units= raw_data[key]['Units'][0][0][0]
        except:
            units= None
        dtype = raw_data[key]['data'][0][0][0][0].dtype
        desc = raw_data[key][0][0][3][0]
        data_def.append([key, len(data),rate, units, dtype, desc, data])
#         features_scrubbed.append(key)
        if rate == 0.25: # sample rate < 1Hz
            #interpolate_to_1hz(data)
            features_25.append(key)
        else: # sample rate > 1Hz
            features_1.append(key)
#     #print len(features_scrubbed)
    return features_25, features_1, data_def

# Create recarray with feature vectors. Normalize all feature data vectors to fit the 1Hz resolution
def feature_vector_array(raw_data, features_25, features_1,file_name):
    arr_dict = {}
    # Setup of final array
    # arr = np.zeros((len(raw_data['SAT']['data'][0][0])),dtype=[('timestamp', np.str_, 20)]) #'datetime64[us]'
    ff = features_1+features_25
    d_length = len(ff)
    t_length = len(raw_data['SAT']['data'][0][0])
    arrs_1d = [0]
    
    # selecting the base timestamp (first record in file) and filling first column of csv with timestamp data based on it
    year       = raw_data['DATE_YEAR']['data'][0][0][0,0]
    month      = raw_data['DATE_MONTH']['data'][0][0][0,0]
    day        = raw_data['DATE_DAY']['data'][0][0][0,0]
    hour       = raw_data['GMT_HOUR']['data'][0][0][0,0]
    minute     = raw_data['GMT_MINUTE']['data'][0][0][0,0]
    sec        = raw_data['GMT_SEC']['data'][0][0][0,0]

    # 687200403251247.mat
    if month <1 or month > 12:
      year = int(file_name[3:7])
      month = int(file_name[7:9])
      day = int(file_name[9:11])
      hour = int(file_name[11:13])
      minute = int(file_name[13:15])
      sec = 0

    base = datetime.datetime(year, month, day, hour, minute, sec)
    date_list = [base + datetime.timedelta(seconds=x) for x in range(0, t_length)]
    dates = np.array(date_list)
#     flghtnum= np.repeat(file_name, len(dates))
    # arr['timestamp'] = dates
    feature_dict['timestamp'] = (dates.T).tolist()
    
    # Filling all feature vector columns in the recarray
    for key in features_1:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = data[0:data.size:rate]
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = (norm_v.T).tolist()[0]
    size_1hz = len(norm_v)
    for key in features_25:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = np.resize(data,(size_1hz,1))
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = (norm_v.T).tolist()[0]
        # print((norm_v.T).tolist()[0])
    return

def feature_dictionary(feature_dict,raw_data,features_25, features_1,file_name):

    # Setup of final array
    # arr = np.zeros((len(raw_data['SAT']['data'][0][0])),dtype=[('timestamp', np.str_, 20)]) #'datetime64[us]'
    ff = features_1+features_25
    d_length = len(ff)
    t_length = len(raw_data['SAT']['data'][0][0])
    arrs_1d = [0]
    
    # selecting the base timestamp (first record in file) and filling first column of csv with timestamp data based on it
    year       = raw_data['DATE_YEAR']['data'][0][0][0,0]
    month      = raw_data['DATE_MONTH']['data'][0][0][0,0]
    day        = raw_data['DATE_DAY']['data'][0][0][0,0]
    hour       = raw_data['GMT_HOUR']['data'][0][0][0,0]
    minute     = raw_data['GMT_MINUTE']['data'][0][0][0,0]
    sec        = raw_data['GMT_SEC']['data'][0][0][0,0]

    # 687200403251247.mat
    if month <1 or month > 12:
      year = int(file_name[3:7])
      month = int(file_name[7:9])
      day = int(file_name[9:11])
      hour = int(file_name[11:13])
      minute = int(file_name[13:15])
      sec = 0

    base = datetime.datetime(year, month, day, hour, minute, sec)
    date_list = [base + datetime.timedelta(seconds=x) for x in range(0, t_length)]
    dates = np.array(date_list)
#     flghtnum= np.repeat(file_name, len(dates))
    # arr['timestamp'] = dates
    feature_dict['timestamp'] =  feature_dict['timestamp'] + (dates.T).tolist()
    
    # Filling all feature vector columns in the recarray
    for key in features_1:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = data[0:data.size:rate]
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = feature_dict[key] + (norm_v.T).tolist()[0]
    size_1hz = len(norm_v)
    for key in features_25:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = np.resize(data,(size_1hz,1))
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = feature_dict[key] + (norm_v.T).tolist()[0]
        # print((norm_v.T).tolist()[0])
    return

def write_output(firstDataFrame,method):
  if method == "CSV":
    firstDataFrame.to_csv("Data_1.csv")
    print("Written Successfully")
  elif method == "JSON":
    results = firstDataFrame.to_json('Data_1.json',orient="split")
    print("Written SucessFully !")


if __name__ == "__main__":
  start = datetime.datetime.now()
  file_list = glob.glob('' + "*.mat")
  feature_dict = {}
  # print(file_list)
  # file_list = ['687200403251247.mat', '687200403251438.mat', '687200403251602.mat', '687200403251723.mat', '687200403260749.mat',]
  # file_list = ['687200403251247.mat']
  for ind,fileName in enumerate(file_list):
      file_name = fileName
      raw_mat_data_dict = open_mat_file(file_name)
      features_25, features_1, data_def = extract_features(raw_mat_data_dict)
      if ind == 0:
        feature_vector_array(raw_mat_data_dict, features_25, features_1,file_name)
      else:
        feature_dictionary(feature_dict,raw_mat_data_dict,features_25, features_1,file_name)
      print("Processed {} file".format(ind))
  # result = pd.DataFrame.from_dict(feature_dict)
      # if ind == 0:
      #   # firstDataFrame = pd.DataFrame.from_records(feature_array)
      #   firstDataFrame = pd.DataFrame.from_dict(feature_array)
      # else:
      #   # firstDataFrame = pd.concat([firstDataFrame,pd.DataFrame.from_records(feature_array)])
      #   firstDataFrame = pd.concat([firstDataFrame,pd.DataFrame.from_dict(feature_array)])
  
  # firstDataFrame.reset_index(inplace = True)
  # write_output(firstDataFrame,method = "JSON")
  end = datetime.datetime.now()
  print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:00.000235


In [23]:
for ind, key in enumerate(key_list):
    start = datetime.datetime.now()
    s3_key = key.key
    flght,set_num,date_s= s3_key.split('.')[0].split('/')
    raw_mat = load_mat_file(s3_key)
    features_25, features_1, data_def= extract_features(raw_mat)
    feature_vector_array(raw_mat, features_25, features_1,date_s)
    end = datetime.datetime.now()
    print("Time Taken to run",flght,set_num, s3_key, str(end-start))
    del(raw_mat)
    if ind ==100:
        break

Time Taken to run Flight 652 1 Flight 652/1/652200101092009.mat 0:00:00.117485
Time Taken to run Flight 652 1 Flight 652/1/652200101092046.mat 0:00:00.094606
Time Taken to run Flight 652 1 Flight 652/1/652200101100441.mat 0:00:00.084753
Time Taken to run Flight 652 1 Flight 652/1/652200101111519.mat 0:00:00.103541
Time Taken to run Flight 652 1 Flight 652/1/652200101111622.mat 0:00:00.289323
Time Taken to run Flight 652 1 Flight 652/1/652200101112303.mat 0:00:00.185951
Time Taken to run Flight 652 1 Flight 652/1/652200101120916.mat 0:00:00.150010
Time Taken to run Flight 652 1 Flight 652/1/652200101121117.mat 0:00:00.096959
Time Taken to run Flight 652 1 Flight 652/1/652200101121118.mat 0:00:00.192247
Time Taken to run Flight 652 1 Flight 652/1/652200101121218.mat 0:00:00.180943
Time Taken to run Flight 652 1 Flight 652/1/652200101121341.mat 0:00:00.206392


In [ ]:
#  Time Taken for open_mat_file Function = 00.21 min
#  Time taken for extract Features = 00.01 min
#  Time Taken for feature_vector_array = 9:00 min

In [ ]:
time_str = []
for v in feature_dict['timestamp']:
  time_str.append(v.strftime("%d-%b-%Y (%H:%M:%S.%f)"))

feature_dict['timestamp'] = time_str

with open('result.json', 'w') as fp:
    json.dump(feature_dict, fp)

In [ ]:

# len(firstDataFrame) is 1619916
# len(feature_dict['FPAC']) is 1619916

In [ ]:
destination_df = firstDataFrame.copy()

In [ ]:
destination_df.head()

In [ ]:
#  Time Taken for open_mat_file Function = 00.21 min
#  Time taken for extract Features = 00.01 min
#  Time Taken for feature_vector_array = 26:00 min
#  Time Taken for Writing = 8 min
# Time Taken = 34 min
raw_mat_data_dict['TRKM']['Rate']

In [ ]:
firstDataFrame.reset_index(inplace=True)

In [ ]:
results = firstDataFrame.to_json('tata.json',orient="split")
results

In [ ]:
start = datetime.datetime.now()
l =[1,2,3]
g = [2,3,4]
l+g
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))